## Settings for Matroos database

The matroos database is the operational forecasting database of Rijkswaterstaat. Several times a day new forecasts are made with several hydrodynamic (tides and surge) models and wave models. 
The most important models in this context are:
- dd zuno-v4 - this is the current model for the southern North-Sea (available 21-april-2020 until now)
- harmonie - the weather model currently used at KNMI

![dd zunov4](https://matroos.deltares.nl/direct/get_matroos.php?source=dcsmv6_zunov4_zuno_kf_harmonie&color=VELUV_ABS&contour=&anal=&from=202305050900&z=0&xmin=-6&xmax=11.6&ymin=49.2&ymax=57.2&smin=0.000&smax=1&size=640&smincolor=blue&smaxcolor=yellow&coords=WGS84&scontour=&vx=25&vy=25&vecsize=1&xn=1&yn=1&vector=&format=png "Zunv4 model")


In [ ]:
# installation on colab
# run this cell to install the required packages, but only once
# You may have to restart the runtime after running this cell (Runtime -> Restart runtime)
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  %pip install parcels cftime netCDF4 cgen zarr Cartopy pymbolic
  # fix from this site https://github.com/googlecolab/colabtools/issues/3134
  # TODO: is this still needed?
  #!pip install importlib-metadata==4.0.1
  #!pip install xarray==0.18.1

In [ ]:
# extenal modules
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
import getpass
import shutil
from parcels import FieldSet, ParticleSet, Variable, JITParticle, ScipyParticle, AdvectionRK4, plotTrajectoriesFile
import math
from datetime import timedelta
import dateutil
from operator import attrgetter
import shutil

In [ ]:
# Settings for Matroos
# use:
# https://matroos.rws.nl if you have an account at that server or
# https://matroos.deltares.nl/ if you have a vpn connection to Deltares or are working in a Deltares office 
database_url = "https://matroos.rws.nl"
print("Enter your Matroos username and password\n")
username = getpass.getpass(prompt="Username:")
password = getpass.getpass(prompt="Password:") #Do not save the password to a public location, such as at github
water_model = "dcsmv6_zunov4_zuno_kf_harmonie"
#print(f'username={username},password={password}')

In [ ]:
# Check the connection to the server
# If you get a data-time for today then everything is working
# There is no need to re-run this cell all the time, since it's just a check

# Download url with get parameters 
url=f'{database_url}/direct/get_anal_times.php?database=maps2d&loc=&source={water_model}&unit=&tstart=&tstop=&timezone=&most_recent=5'

# send request to server
r = requests.get(url, auth=(username, password))
print(f'response = {r}  [200 is good]\n')
temp_file = 'temporary_file_with_times.csv'
open(temp_file, 'wb').write(r.content)
# download data into a pandas dataframe
df = pd.read_csv(temp_file, sep=';', index_col=0, parse_dates=True, dayfirst=True)
os.remove(temp_file)
df

In [ ]:
# Settings for collecting the data
#initial and final date and time in timezone UTCC (NL winter time is UTC+1, NL summer time is UTC+2)
tstart = "2022-10-01 00:00:00" # start of the period of interest (year-month-day hour:minute:second)
tstop  = "2022-10-02 00:00:00" # end of the period of interest
longitude_min = 4.0 # minimum longitude (most western point in degrees)
longitude_max = 6.0 # maximum longitude (most eastern point in degrees)
latitude_min = 53.0 # minimum latitude (most southern point in degrees)
latitude_max = 54.0 # maximum latitude (most northern point in degrees)
cellx = 100 # number of data points in x-direction (larger will give more detail, but also result in larger files)
celly = 100 # number of data points in y-direction ( you should start carefully with 100, and use an image to check if the data has sufficient detail; see image below)
# The data has the form of a cube with directions x, y and time. The size of the cube is (cellx,celly,ntime) and proportional in filesize to the prduct of these three numbers. For larger values the database will become slower and beyond a certain size it will not respond anymore.


# no need to change anything below this line
fieldoutput="VELU,VELV,VELUV_ABS,SEP" #download these variables (velocity and waterlevel)
stridetime=1 # collect all times or skip some (1 means all times, 2 means every second time, 3 means every third time, etc.)

#reformat time from for example "2022-10-01 00:00:00" to "202210010000"
tstart_str = tstart.replace("-","").replace(" ","").replace(":","")
tstop_str = tstop.replace("-","").replace(" ","").replace(":","")


In [ ]:
# Download the data from the server

#if folder data does not exist, create it
if not os.path.exists("data"):
    os.makedirs("data")

#compose url
url = database_url + "/direct/get_matroos.php?source=" + water_model + "&anal=000000000000&z=0&xmin=" + str(longitude_min) + "&xmax=" + str(longitude_max) + "&ymin=" + str(latitude_min) + "&ymax=" + str(latitude_max) + "&coords=WGS84&xmin_abs=" + str(longitude_min) + "&xmax_abs=" + str(longitude_max) + "&ymin_abs=" + str(latitude_min) + "&ymax_abs=" + str(latitude_max) + "&color=" + fieldoutput + "&interpolate=size&now=" + tstop_str + "&to=" + tstop_str + "&from=" + tstart_str + "&outputformat=nc&stridex=&stridey=&stridetime=" + str(stridetime) + "&xn=" + str(cellx) + "&yn=" + str(celly) + "&celly=" + str(celly) + "&cellx=" + str(cellx) + "&fieldoutput=" + fieldoutput + "&format=nc"
print(f'url={url}\n')
# send request to server
print("Be patient, this can take a while")
r = requests.get(url, auth=(username, password))
print(f'response = {r}  [200 is good]\n')
# save data to file
filename = "data/" + water_model + "_" + tstart_str + "_" + tstop_str + ".nc"
f=open(filename, "wb")
f.write(r.content)
f.close()


In [ ]:
# Plot the last time step of the data

# open data file with xarray
filename = "data/" + water_model + "_" + tstart_str + "_" + tstop_str + ".nc"
ds = xr.open_dataset(filename)
x=ds['x'][:]
y=ds['y'][:]
uv_abs=ds['VELUV_ABS'][-1,:,:]
fig, ax = plt.subplots(figsize=(10, 6))
p1=ax.pcolormesh(x,y,uv_abs) # plot last time step of velocity magnitude
fig.colorbar(p1)
stide=10 # plot every stide-th arrow
ax.quiver(x[::stide],y[::stide],ds['VELU'][-1,::stide,::stide],ds['VELV'][-1,::stide,::stide],scale=10) 
plt.title(f'Currents at time {ds["time"][-1].values}')
#ds 

## Releasing particles in the field

In [ ]:
# settings for the particle tracking
water_filename = "data/" + water_model + "_" + tstart_str + "_" + tstop_str + ".nc"
particle_filename="ZUNO_Particles.zarr"
initial_longitude = 5.0 # longitude of the initial position of the particle
initial_latitude = 53.5 # latitude of the initial position of the particle
initial_time = tstart # time of the initial position of the particle (start of data is tstart, or give explicit time in format "2022-10-01 00:00:00")
output_timestep = timedelta(hours=1) # time step of the output
dt=timedelta(minutes=5) # time step of the numerical computation
simulation_time = timedelta(days=1) # total simulation time

In [ ]:
# Compute the particle trajectories

# remove particle folder if it alreadt exists
if os.path.exists(particle_filename):
  shutil.rmtree(particle_filename)

# open data file with xarray
ds_water = xr.open_dataset(water_filename)
# create fieldset (input data for the particle tracking)
variables = {'U': 'VELU',
             'V': 'VELV'}
dimensions = { 'U': {'lat': 'y', 'lon': 'x', 'time': 'time'}, 'V': {'lat': 'y', 'lon': 'x', 'time': 'time'} }
#fieldset = FieldSet.from_netcdf(filename, variables, dimensions, allow_time_extrapolation=True)
fieldset = FieldSet.from_xarray_dataset(ds_water,variables,dimensions)

# create particles
t0=dateutil.parser.parse(initial_time)
pset = ParticleSet.from_list(fieldset=fieldset,       # the fields on which the particles are advected
                             pclass=JITParticle,      # the type of particles (JITParticle or ScipyParticle)
                             lon=[initial_longitude], # release longitude
                             lat=[initial_latitude],  # release latitude
                             time=[t0])     # release time

# run the particle advection
particle_file=pset.ParticleFile(name=particle_filename, outputdt=output_timestep)
pset.execute(AdvectionRK4,
             runtime=simulation_time,
             dt=dt,
             output_file=particle_file
            )

#pset.show()

particle_file.close()

In [ ]:
ds_part= xr.open_dataset('ZUNO_Particles.zarr', engine='zarr')

%matplotlib inline
plt.figure(figsize=(15,10))

(ds.VELUV_ABS[0]).plot.pcolormesh()
plt.scatter(ds_part.lon.T,ds_part.lat.T,c=ds_part.time.T,cmap=plt.cm.jet)